# AceleraDev DataScience 

## Setup

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data




In [1]:
#lendo os pacotes
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
y_train = df['SalePrice']
X_train = df.drop(columns = 'SalePrice')

In [4]:
X_test = pd.read_csv('../data/test.csv')
y_test = pd.read_csv('../data/sample_submission.csv')

## Analysis

### Selecao por completude

In [5]:
#Criando um dataframe auxliar para analisar a consistencia das variaveis
cons = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'missing' : df.isna().sum(),
                    'size' : df.shape[0],
                    'unicos': df.nunique()})
cons['percentual'] = round(cons['missing'] / cons['size'],2)

### Removendo colunas com dados missing

In [6]:
print('Contagem de colunas com ATÉ 20% de dados faltantes', cons[cons.percentual < 0.2].shape[0])
print('Contagem de colunas com 0% de dados faltantes',  cons[cons.percentual == 0].shape[0])

Contagem de colunas com ATÉ 20% de dados faltantes 76
Contagem de colunas com 0% de dados faltantes 63


In [7]:
cons[cons.percentual == 0]['tipo'].value_counts()

int64     35
object    28
Name: tipo, dtype: int64

In [8]:
cons['completa'] = ['completa' if x == 0 else 'faltante' for x in cons['percentual']]

In [9]:
mantem = list(cons[cons['completa'] == 'completa']['colunas'])
df = df[mantem]

In [10]:
df.drop

<bound method DataFrame.drop of         Id  MSSubClass MSZoning  LotArea Street LotShape LandContour  \
0        1          60       RL     8450   Pave      Reg         Lvl   
1        2          20       RL     9600   Pave      Reg         Lvl   
2        3          60       RL    11250   Pave      IR1         Lvl   
3        4          70       RL     9550   Pave      IR1         Lvl   
4        5          60       RL    14260   Pave      IR1         Lvl   
...    ...         ...      ...      ...    ...      ...         ...   
1455  1456          60       RL     7917   Pave      Reg         Lvl   
1456  1457          20       RL    13175   Pave      Reg         Lvl   
1457  1458          70       RL     9042   Pave      Reg         Lvl   
1458  1459          20       RL     9717   Pave      Reg         Lvl   
1459  1460          20       RL     9937   Pave      Reg         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0       AllPub    Inside

In [11]:
colunas_numericas = list(cons[((cons['tipo'] != 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])
colunas_numericas.remove('SalePrice')

In [12]:
colunas_categoricas = list(cons[((cons['tipo'] == 'object') &
                              (cons['completa'] == 'completa'))]['colunas'])

## Categoricas

### Label Encoding

In [13]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'SalePrice'],
      dtype='object')

### Label encoder

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
encoding = LabelEncoder()

In [16]:
encoding.fit_transform(df['TotRmsAbvGrd'])

array([6, 4, 4, ..., 7, 3, 4], dtype=int64)

### Dummies

In [17]:
df.shape

(1460, 63)

In [18]:
pd.get_dummies(df).shape

(1460, 222)

### Catboost

In [21]:
import category_encoders as ce

In [23]:
df[colunas_categoricas].head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,PConc,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,CBlock,GasA,Ex,Y,SBrkr,TA,Typ,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,PConc,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,BrkTil,GasA,Gd,Y,SBrkr,Gd,Typ,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,PConc,GasA,Ex,Y,SBrkr,Gd,Typ,Y,WD,Normal


In [24]:
catb = ce.CatBoostEncoder(cols=colunas_categoricas)

In [25]:
catb

CatBoostEncoder(a=1,
                cols=['MSZoning', 'Street', 'LotShape', 'LandContour',
                      'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                      'ExterQual', 'ExterCond', 'Foundation', 'Heating',
                      'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
                      'Functional', 'PavedDrive', 'SaleType', 'SaleCondition'],
                drop_invariant=False, handle_missing='value',
                handle_unknown='value', random_state=None, return_df=True,
                sigma=None, verbose=0)

In [35]:
aux = catb.fit_transform(X_train[colunas_categoricas], y_train, random_state=42)
aux.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Foundation,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,PavedDrive,SaleType,SaleCondition
0,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,...,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890,180921.195890
1,194710.597945,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,180921.195890,180921.195890,194710.597945,...,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945,180921.195890,194710.597945,194710.597945,194710.597945,194710.597945
2,190307.065297,190307.065297,180921.195890,190307.065297,190307.065297,194710.597945,190307.065297,194710.597945,194710.597945,190307.065297,...,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297,194710.597945,190307.065297,190307.065297,190307.065297,190307.065297
3,198605.298973,198605.298973,202210.597945,198605.298973,198605.298973,180921.195890,198605.298973,180921.195890,204307.065297,198605.298973,...,180921.195890,198605.298973,180921.195890,198605.298973,198605.298973,204307.065297,198605.298973,198605.298973,198605.298973,180921.195890
4,186884.239178,186884.239178,181473.731963,186884.239178,186884.239178,181210.597945,186884.239178,180921.195890,188230.298973,186884.239178,...,204307.065297,186884.239178,198605.298973,186884.239178,186884.239178,188230.298973,186884.239178,186884.239178,186884.239178,198605.298973


In [36]:
X_train['MSZoning'].dtype, aux['MSZoning'].dtype

(dtype('O'), dtype('float64'))

In [37]:
X_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


## Numericas

### Standard Scaler

In [38]:
from sklearn.preprocessing import StandardScaler

In [39]:
std = StandardScaler()

In [40]:
std.fit_transform(X_train[colunas_numericas])

array([[-1.73086488,  0.07337496, -0.20714171, ..., -0.08768781,
        -1.5991111 ,  0.13877749],
       [-1.7284922 , -0.87256276, -0.09188637, ..., -0.08768781,
        -0.48911005, -0.61443862],
       [-1.72611953,  0.07337496,  0.07347998, ..., -0.08768781,
         0.99089135,  0.13877749],
       ...,
       [ 1.72611953,  0.30985939, -0.14781027, ...,  4.95311151,
        -0.48911005,  1.64520971],
       [ 1.7284922 , -0.87256276, -0.08016039, ..., -0.08768781,
        -0.8591104 ,  1.64520971],
       [ 1.73086488, -0.87256276, -0.05811155, ..., -0.08768781,
        -0.1191097 ,  0.13877749]])

### MinMaxScaler

In [43]:
from sklearn.preprocessing import MinMaxScaler

In [44]:
minmax = MinMaxScaler()

In [45]:
minmax.fit_transform(X_train[colunas_numericas])

array([[0.00000000e+00, 2.35294118e-01, 3.34198042e-02, ...,
        0.00000000e+00, 9.09090909e-02, 5.00000000e-01],
       [6.85400960e-04, 0.00000000e+00, 3.87950174e-02, ...,
        0.00000000e+00, 3.63636364e-01, 2.50000000e-01],
       [1.37080192e-03, 2.35294118e-01, 4.65072799e-02, ...,
        0.00000000e+00, 7.27272727e-01, 5.00000000e-01],
       ...,
       [9.98629198e-01, 2.94117647e-01, 3.61868705e-02, ...,
        1.61290323e-01, 3.63636364e-01, 1.00000000e+00],
       [9.99314599e-01, 0.00000000e+00, 3.93418869e-02, ...,
        0.00000000e+00, 2.72727273e-01, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 4.03701886e-02, ...,
        0.00000000e+00, 4.54545455e-01, 5.00000000e-01]])

### MinAbsScaler

In [46]:
from sklearn.preprocessing import MaxAbsScaler

In [47]:
maxabs = MaxAbsScaler()

In [48]:
maxabs.fit_transform(X_train[colunas_numericas])

array([[6.84931507e-04, 3.15789474e-01, 3.92575902e-02, ...,
        0.00000000e+00, 1.66666667e-01, 9.99004975e-01],
       [1.36986301e-03, 1.05263158e-01, 4.46003391e-02, ...,
        0.00000000e+00, 4.16666667e-01, 9.98507463e-01],
       [2.05479452e-03, 3.15789474e-01, 5.22660224e-02, ...,
        0.00000000e+00, 7.50000000e-01, 9.99004975e-01],
       ...,
       [9.98630137e-01, 3.68421053e-01, 4.20079444e-02, ...,
        1.61290323e-01, 4.16666667e-01, 1.00000000e+00],
       [9.99315068e-01, 1.05263158e-01, 4.51439058e-02, ...,
        0.00000000e+00, 3.33333333e-01, 1.00000000e+00],
       [1.00000000e+00, 1.05263158e-01, 4.61659969e-02, ...,
        0.00000000e+00, 5.00000000e-01, 9.99004975e-01]])